Вариант 1
Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы получаем должность) с сайтов HH(обязательно) и/или Superjob(по желанию). Приложение должно анализировать несколько страниц сайта (также вводим через input или аргументы). Получившийся список должен содержать в себе минимум:
Наименование вакансии.
Предлагаемую зарплату (разносим в три поля: минимальная и максимальная и валюта. цифры преобразуем к цифрам).
Ссылку на саму вакансию.
Сайт, откуда собрана вакансия.
По желанию можно добавить ещё параметры вакансии (например, работодателя и расположение). Структура должна быть одинаковая для вакансий с обоих сайтов. Общий результат можно вывести с помощью dataFrame через pandas. Сохраните в json либо csv.

In [44]:
import json
import requests
from bs4 import BeautifulSoup as bs

#load job vacancy data from hh.ru
def load_hh(vacancy):
    vacancy = vacancy.replace(' ', '+')
    url = 'https://hh.ru/search/vacancy?text=' + vacancy
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36'}
    response = requests.get(url, headers=headers)
    soup = bs(response.text, 'lxml')

    vacancy_items = soup.find_all('div', {'class': 'vacancy-serp-item-body'})
    vacancy_data = []

    for item in vacancy_items:
        vacancy_name = item.find('a', {'class': 'serp-item__title'}).getText()
        vacancy_link = item.find('a', {'class': 'serp-item__title'})['href']
        vacancy_salary = item.find('span', {'class': 'bloko-header-section-3'})
        if vacancy_salary is not None:
            vacancy_salary_currency = vacancy_salary.getText().split(" ")[-1]
            vacancy_salary = vacancy_salary.getText().replace('\xa0', '').replace('\u202f', '').replace(' ', '')
            if vacancy_salary.find('от') != -1:
                vacancy_salary_min = int(vacancy_salary.replace('от', '').replace(vacancy_salary_currency, ''))
                vacancy_salary_max = None
            elif vacancy_salary.find('до') != -1:
                vacancy_salary_min = None
                vacancy_salary_max = int(vacancy_salary.replace('до', '').replace(vacancy_salary_currency, ''))
            else:
                vacancy_salary_min = int(vacancy_salary.split('–')[0])
                vacancy_salary_max = int(vacancy_salary.split('–')[1].replace(vacancy_salary_currency, ''))
        else:
            vacancy_salary_min = None
            vacancy_salary_max = None
            vacancy_salary_currency = None
        vacancy_data.append({
            'name': vacancy_name,
            'link': vacancy_link,
            'salary_min': vacancy_salary_min,
            'salary_max': vacancy_salary_max,
            'currency': vacancy_salary_currency,
            'source': 'hh.ru'
        })
    return vacancy_data

vacancy = input('Введите название вакансии: ')
with open(vacancy +'_hh.json', 'w') as file:
    json.dump(load_hh(vacancy), file, ensure_ascii=False)
print("done")

done
